In [52]:
from collections import namedtuple

from vega_sim.null_service import VegaServiceNull
import vega_sim.proto.vega as vega_protos


In [53]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])

MM_WALLET = WalletConfig("mm", "mm")

TRADER_a1_WALLET = WalletConfig("a1", "pin")

TRADER_a2_WALLET = WalletConfig("a2", "pin")

TRADER_2_WALLET = WalletConfig("T2", "pin")

TRADER_3_WALLET = WalletConfig("T3", "pin")

TERMINATE_WALLET = WalletConfig("Settle", "pin")

wallets = [MM_WALLET, TRADER_2_WALLET, TRADER_3_WALLET, TRADER_a1_WALLET , TRADER_a2_WALLET, TERMINATE_WALLET]


In [54]:
vega = VegaServiceNull(run_with_console=False, use_full_vega_wallet=False)
vega.start()

INFO:vega_sim.null_service:Running NullChain from vegahome of /var/folders/t8/z7v3pdr57d1_rrxyb2skvdtm0000gn/T/tmp9bgaewet
INFO:vega_sim.null_service:Launching GraphQL node at port 52245
INFO:vega_sim.null_service:Launching Console at port 52251
Using function with raw data from data-node VegaService.all_markets. Be wary if prices/positions are not converted from int form


In [55]:
for wallet in wallets:
    vega.create_wallet(wallet.name, passphrase=wallet.passphrase)
vega.forward('10s')

In [56]:
vega.mint(
    MM_WALLET.name,
    asset="VOTE",
    amount=10000,
)

vega.forward("20s")
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
    max_faucet_amount=10e15,  # is this including decimals or not?
)

tdai_id = vega.find_asset_id(symbol="tDAI")

mint_amount = {MM_WALLET.name: 10e12, TRADER_a1_WALLET.name: 10e12, TRADER_a2_WALLET.name: 10e12, TRADER_2_WALLET.name: 2000, TRADER_3_WALLET.name: 90}

vega.mint(
    MM_WALLET.name,
    tdai_id,
    amount=mint_amount[MM_WALLET.name]
)
vega.mint(
    TRADER_a1_WALLET.name,
    tdai_id,
    amount=mint_amount[TRADER_a1_WALLET.name]
)
vega.mint(
    TRADER_a2_WALLET.name,
    tdai_id,
    amount=mint_amount[TRADER_a2_WALLET.name]
)
vega.mint(
    TRADER_2_WALLET.name,
    tdai_id,
    amount=mint_amount[TRADER_2_WALLET.name]
)
vega.mint(
    TRADER_3_WALLET.name,
    tdai_id,
    amount=mint_amount[TRADER_3_WALLET.name]
)
vega.wait_for_total_catchup()


In [57]:
# Create a market
vega.create_simple_market(
        market_name="BTC:DAI_Mar22",
        proposal_wallet=MM_WALLET.name,
        settlement_asset_id=tdai_id,
        termination_wallet=TERMINATE_WALLET.name,
        position_decimals=0,
        market_decimals=0,
        risk_aversion=0.000001,
        tau=0.1,
        sigma=1,
        
)

market_id = vega.all_markets()[0].id

Using function with raw data from data-node VegaService.all_markets. Be wary if prices/positions are not converted from int form


In [58]:
# submit liquidity
vega.submit_simple_liquidity(
    wallet_name=MM_WALLET.name,
    market_id=market_id,
    commitment_amount=100000,
    fee=0.001,
    reference_buy="PEGGED_REFERENCE_BEST_BID",
    reference_sell="PEGGED_REFERENCE_BEST_ASK",
    delta_buy=55,
    delta_sell=55,
    is_amendment=False,
)

vega.forward('1s')
vega.forward('1s')

In [59]:
# Post GTC limit order
buy_order_id = vega.submit_order(
    trading_wallet=TRADER_a2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=5,
    price=5,
    wait=True,
)

sell_order_id = vega.submit_order(
    trading_wallet=TRADER_a1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=10,
    price=1000,
    wait=True,
)

buy_order_id = vega.submit_order(
    trading_wallet=TRADER_a2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=10,
    price=10,
    wait=True,
)

sell_order_id = vega.submit_order(
    trading_wallet=TRADER_a1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=10,
    price=10,
    wait=True,
)


In [63]:
# Check order status/ market state

for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))
    print(vega.margin_levels(wallet.name))
    print(general+margin+bond - mint_amount)
    print(vega.order_status(order_id=buy_order_id))

print(vega.market_data(market_id=market_id))

mm General = 9999999515650.46
mm Margin  = 384349.53984
mm Bond    = 100000.0
[MarginLevels(maintenance_margin=320291.2832, search_level=352320.41152, initial_margin=384349.53984, collateral_release_level=448407.79648, party_id='4bc6835da01db9716c54d976e698faa96e8c14131eda8bdfef9a41ff9af9f283', market_id='43994fe1d1b2b1542f009dffd84a864fd6f18677fe56510588a361cb426439a5', asset='f3c80514ca9b47a93a9fec90b8ebcbe35890898c09e2ca4120aab6a18b93392d', timestamp=1638185346259066000)]


TypeError: unsupported operand type(s) for -: 'float' and 'dict'

In [ ]:
buy_order_id = vega.submit_order(
    trading_wallet=TRADER_2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=40,
    price=50,
    wait=True,
)
for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))

mm General = 9999999891463.432
mm Margin  = 8536.56862
mm Bond    = 0.0
T2 General = 1615.65046
T2 Margin  = 384.34954
T2 Bond    = 0
T3 General = 90.0
T3 Margin  = 0
T3 Bond    = 0
a1 General = 9999999987266.344
a1 Margin  = 12733.65686
a1 Bond    = 0
a2 General = 9999999999795.87
a2 Margin  = 204.13108
a2 Bond    = 0
Settle General = 0
Settle Margin  = 0
Settle Bond    = 0


In [ ]:
# Amend order
vega.amend_order(
    trading_wallet=TRADER_1_WALLET.name,
    market_id=market_id,
    order_id=buy_order_id,
    price=75,
    volume_delta=2,
)



NameError: name 'TRADER_1_WALLET' is not defined

In [ ]:
# Check order status/ market state after amend
print(vega.order_status(order_id=buy_order_id))

print(vega.market_data(market_id=market_id))